# 背景
生成モデルでオキシムエステルの生成を行いましたが、数がおおい

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("oxime_with_keton.csv", index_col=0)
df.head()

,Smiles,T1,S1,S1_osc,similarity,Scaffold,oxime_num,keton
0,CC(=O)O\N=C/c1ccc(Cl)cc1,2.627373,4.566069,0.879576,1.000000,c1ccccc1,1,NaN
1,C/C(=N/OC(=O)c1ccccc1)c1ccccc1,2.830630,4.727700,0.575377,1.000000,O=C(ON=Cc1ccccc1)c1ccccc1,1,CC(=O)c1ccccc1
2,O=C(O/N=C/c1ccccc1)C(=O)ON=C(c1ccccc1)C(c1cccc...,2.487950,4.363856,0.972207,0.536585,O=C(ON=C(c1ccccc1)C(c1ccccc1)c1ccccc1)C(=O)O/N...,2,O=C(c1ccccc1)C(c1ccccc1)c1ccccc1
3,CCCCCC(/C(=N/OC(=O)C)c1ccc(Cl)cc1)=N\OC(=O)C,2.504756,4.149163,0.506938,1.000000,c1ccccc1,2,CCCCCC(=O)C(=O)c1ccc(Cl)cc1
4,C/C(CC(=O)O/N=C\c1ccccc1)=N\OC(C)=O,2.540638,4.326961,0.580047,1.000000,c1ccccc1,2,CC(=O)CC(=O)O/N=C\c1ccccc1


In [3]:
print(list(df.columns))

['Smiles', 'T1', 'S1', 'S1_osc', 'similarity', 'Scaffold', 'oxime_num', 'keton']
